# Note on using this notebook
## Run all cells now before scrolling further. Top Menu > Run > Run all cells

I recommend running all cells as soon as the notebook is opened. Due to the nature of the interactive widgets, it is not possible to save the state, so the notebook is saved without output. If you are perusing the full document, each cell should have run and generated output by the time you get to it. This applies whether viewing locally or via Binder. 

In [ ]:
import ForestForTheTrees as ft

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.inspection import plot_partial_dependence

import altair as alt
import ipywidgets as wid

alt.data_transformers.enable('default')
alt.data_transformers.disable_max_rows()

#seed for reproducibility
np.random.seed = 15

# Visual Explanations for Gradient Boosting Models

## Goal

This notebook explores the interpretation of machine learning models, and how we can generate model explanations that inform human sensemaking. It will particularly focus on Gradient Boosting Regressors, a powerful model often used with tabular data. We'll also use a data visualization to "watch" the model build its own internal structure. The goal is to help the curious reader develop an intuition for how models make predictions, and to investigate ways that we can extract that knowledge for ourselves. 

This notebook is aimed at an audience that has some exposure to data analysis, but may have limited expertise in machine learning. This is also an interactive notebook, so you will be able to use sliders, dropdowns, and other widgets to modify the visualization to further develop your intuition. 

You may notice that this notebook is fairly light on code, and makes most of its calls to a library called ForestForTheTrees. A fair amount of data manipulation goes into visualizing these models, but the point here is not to focus on the details of the code. For those who are interested, the code is contained in the ForestForTheTrees.py file, which can be found in the same directory as this notebook.

This notebook was developed using Python and the Python data science stack, particularly [numpy](https://numpy.org/), [pandas](https://pandas.pydata.org/), and [scikit-learn](https://scikit-learn.org/stable/). [Altair](https://altair-viz.github.io/) was used for data visualization.

## Brief Introduction to Machine Learning

If you're not a data scientist, you might be most familiar with machine learning from the cool and creepy things it does that get coverage on news and social media. Things like [helping doctors diagnose breast cancer](https://ai.googleblog.com/2018/04/an-augmented-reality-microscope.html), [making new celebrity faces](https://research.nvidia.com/publication/2017-10_Progressive-Growing-of), [watching us from the skies](https://www.goodreads.com/book/show/40796190-eyes-in-the-sky?ac=1&from_search=true) and [potentially sending you to prison](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing). While this is not a thinkpiece on the future of AI, I think it's fair to say that it would be easy to look at the news and say that machine learning's primary business is to slowly supplant humans in every meaningful endeavor.

And certainly, anyone reviewing the latest machine learning journals would find plenty of fodder for the belief that researchers are working hard to remove humans, even developers, from the equation. How else to explain research into [automated data cleaning](https://dl.acm.org/citation.cfm?id=2750549), [automated feature extraction](https://arxiv.org/abs/1706.00327) and [automatic relationship extraction from databases](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.24.6356&rep=rep1&type=pdf)?

As a professional data scientist, though, this is pretty far from my experience of how machine learning is applied in practice. In fact, I think that one of the best uses for machine learning is as a complement and support for human thinking. One of the forms that can take is automated insight extraction. Let's motivate this with an example. 

Imagine we are data analysts for a bike sharing company, and we have a dataset with each day from the past year, with the total number of bikes rented each hour of the day. A sample of the dataset is below - each row represents one hour (see the Hour of Day column). 

In [ ]:
f2t = ft.ForestForTheTrees(dataset = "bike", sample_size = None, num_tiles = 40, quantiles = False, learning_rate = 1.)
f2t.build_base_model(300)
f2t.extract_components(True, False)
f2t.explain(.95, None)
bike_dataset = f2t.get_dataset("bike")
bike_df = pd.DataFrame(
    np.hstack((bike_dataset["x"],np.array(bike_dataset["y"]).reshape(-1,1))),
    columns = bike_dataset["feature_names"] + ["Total Bikes Rented"]
)
bike_df.head()

We want to learn something that will help us predict usage for the next month, perhaps to model how to grow our fleet of bikes. The way we would usually do this is to build a machine learning model that models the relationship between facts we know (temperature that day, weather that day, day of the week, whether there was a big public festival or not) and something we know for past days, but not future ones (# of rentals). But, with our many years in the bike-sharing industry, we want to see if we can use our domain knowledge to do better. 

We certainly know many facts that are relevant to making the prediction. For example, people rent more bikes on sunny days than rainy days! But here we run into trouble. To actually make a prediction, we need to be able to calculate a precise number for a rainy Tuesday in November, and a sunny Saturday in April during the annual dogwood festival. How can we do this? We might have a sense that the average number of bikes rented on sunny days is around 3000, and the number of rainy days is about 1500. But you would be very hard-pressed to quantify the exact relationship between sunny and rainy. For example, rainy days tend to be colder. Maybe it's the cold that drives down ridership, and if two days have identical temperatures but one is rainy and the other sunny, ridership will be about the same. Not to mention rain is more common during certain parts of the year, and rain probably affects ridership differently on weekdays vs. weekends, and you can see we have a big mess. These issues (which are referred to as *collinearity* and *interaction effects*) are not our main concern here, but we should note that prediction is a [very hard](https://www.apa.org/pubs/journals/releases/xap-0000040.pdf) science, confounding even those with deep domain knowledge.

However, one of the simplest machine learning models, linear regression, can calculate this in a second.

In [ ]:
#linear regression
naive_linear_regression = LinearRegression()
naive_linear_regression.fit(bike_df.iloc[:, :-1], bike_df.iloc[:,-1])
#naive_linear_regression.score(bike_df.iloc[:, :-1], bike_df.iloc[:,-1])
for x in zip(bike_dataset["feature_names"], naive_linear_regression.coef_):
    print("When {} increases by 1, ridership {} by {}".format(
        x[0],
        "increases" if x[1] > 0 else "decreases",
        round(abs(x[1]),1)
    ))

We get a coefficient that tells us that when it rains, ridership drops by 24.7. Or that for every degree increase of temperature, ridership increases by 1. These conclusions are overly simplistic (and it's generally incorrect to interpret the coefficients this literally), but our model can give us this number and tell us that it is the best possible one (given the limitations of the dataset we give it, and the expressiveness of the model). We can also use a much more powerful and complex model, such as a Gradient Boosting Regressor, and see what it found. This model doesn't give us a nice set of facts, but we can extract something better with a chart called a [Partial Dependence Plot](https://christophm.github.io/interpretable-ml-book/pdp.html). Essentially, a PDP calculates how a prediction changes as we move only one value up and down from the minimum to the maximum. You can see the PDP plot for temperature's effect on ridership below, from 19$^{\circ}$-102$^{\circ}$F.

In [ ]:
#GBR and 1d PDP
naive_gbr = GradientBoostingRegressor()
naive_gbr.fit(bike_df.iloc[:, :-1], bike_df.iloc[:,-1])
def view_pdp(feature_name):
    plot_partial_dependence(
        naive_gbr, 
        bike_df.iloc[:, :-1],
        features = [feature_name],
        feature_names = bike_dataset["feature_names"]
    )

pdp_feature_selector = wid.Dropdown(
    options = bike_dataset["feature_names"],
    value = "Temperature (F)",
    description = 'Select Feature',
)

wid.interact(
    view_pdp,
    feature_name = pdp_feature_selector
)

We can now see that the reality was far more complex than our linear regression indicated. Temperature has a *non-linear* effect on ridership, in that there's not a steady, uniform increase. Rather, at some points in the range, rising temperature means more riders, and in other places, it means less. And ridership rises and falls at different rates in the range. We can no longer succinctly express the effect of temperature in a number, but we now have this curve, which is the most parsimonious (without summarizing and losing information) representation of the actual behavior. 

Before we move on, take the opportunity to familiarize yourself with the dataset by checking out the PDP for every feature. Do the curves match your expectations?

It should be clear that domain expertise won't get us this partial dependence curve. We wouldn't have a lot of luck drawing it out. This is something that computers are going to be able to extract far more efficiently than humans, furious optimizers that they are. However, we need to consider another aspect as well.

## When the Machine is Wrong

Much of the promise and peril of machine learning comes into focus when it is applied to problems that involve human welfare. [Caruana et al](https://dl.acm.org/citation.cfm?id=2788613) relate a story of this kind of moral hazard in one of their works on interpretable models. A prior [study](https://www.sciencedirect.com/science/article/pii/S1532046405000225) of patients with pneumonia investigated several types of model for predicting a patient's pneumonia risk, with the goal of triaging care efficiently. In one model, they discovered a surprising relationship - the model treated patients with asthma as lower risk! Yet asthma was well known by the doctors as an aggravating factor, so much so that patients with the condition generally received aggressive treatment.

Can you see what happened? The "ASTHMA" variable in the dataset didn't just track whether the patient had the condition, but also whether they received aggressive treatment to prevent pneumonia. While the former increased risk, the latter naturally reduced it.

What would have happened if this insight had not been uncovered? The researchers also trained a multi-task neural net, which predicted patient risk more accurately but was essentially a black box. Given that it almost certainly learned the same asthma => lower risk relationship as the logistic regression, using this model in practice could have cost lives by invisibly assinging asthmatics an inappropriate level of care.

We see here the critical value of domain knowledge. First, the researchers were able to compare the insights from the model against their intuitions, and act when something was found to be surprising. Second, they were able to reason about the MEANING of a feature, and refine their understanding of how it should actually be used. In this case, simple optimization of a loss function was insufficient to find the model which would work best in practice.

## Machine Learning For Insight Extraction

At this point, we have a better sense of the relative merits of human and machine analysis. Broadly, machines can find the thresholds, decision boundaries, line slopes, and other "facts" that are optimal for our dataset. They can assemble these into specific predictions. But, because humans understand the meaning of data in its real-world context, and have a rich tapestry of experience, we have a good sense of generally what kind of facts we SHOULD find. We can be surprised, and can be wrong, but know to question things that seem suspicious. Each complements the other.

The answer might seem obvious - computers should build data models, people should inspect, guide, and review them, and we'll end up with high quality data analysis. We can term this "automated insight extraction" - analysts build models not because they should be used for automated predictions (at least right off the bat), but because they extract facts far more efficiently than people. 

The problem is that computers broadly and models specifically are terrible at telling us what they know. Models do not store a list of neat facts that we can just read. Instead, their internals consist of matrices that represent more abstract concepts such as decision trees, artificial neurons, weights and bias terms, etc. Techniques like the partial dependence plots above have to be applied as a secondary process to generate something human-readable out of the model. They have known weaknesses, in which they [fail](https://arxiv.org/abs/1904.00561) to extract everything the model "knows", or introduce misleading [distortions](https://arxiv.org/pdf/1905.03151.pdf).

The purpose of this notebook is to pick one complex model type (Gradient Boosting models), make some simplifying restrictions, and then present interactive visual techniques to extract as much of what the model "knows" as possible into a form that humans can understand. We'll also look at a method for quantifying how much of that potential insight we actually got out. That will let us make explicit tradeoffs between complexity and fidelity to the underlying model, so that we can construct a situation- and audience-specific explanation. But first, let's discuss exactly what Gradient Boosting is.

## Tree-based Models

Gradient Boosting models are a type of tree ensemble. Let's unpack these two terms. Tree models are based on the underlying concept of a decision tree, which has a long [history](https://stats.stackexchange.com/questions/257537/who-invented-the-decision-tree) in statistics, operations research, philosophy, and finally as a machine learning model. In fact, we have all probably made a decision tree at some point for use in our own lives.

A decision tree is simply a method for making a decision composed of if -> then rules and possible outcomes. "If it rains, then I will stay home, otherwise I will go to the market" is a simple decision tree. It is "1-deep" or "1-layer" since there is only one decision point (also called a decision stub). A 2-layer decision tree would look like this:

If it rains, then check the pantry for bread. If there is no bread, then go to the store, otherwise stay home. If it's sunny, then check the temperature. If the temperature is over 30F, then go to the market, otherwise stay home.

What turns this simple form of human decisionmaking into a machine learning model are four modifications:

1. Use variables from a dataset for both the decision points and outcomes, rather than vague real-world events.
2. Write it in a logical form, using these same variable names.
3. Choose the "thresholds" and order of decisions based on an optimization algorithm.
4. Rather than trying to make a single future decision, and then most likely discarding this particular decision tree, optimize based on past data so that the decision tree best "predicts" past behavior. This changes the task from "Lucas chooses whether he goes to the market today" to "Lucas's friend Ella tries to guess whether he will go to the market today based on data she has collected about his past whereabouts". (I promise ML isn't all creepy, but yeesh!)

From this change, our model could now be written in pseudocode as:
```
def predict_lucas_at_market():
    IF WEATHER=="RAIN":
        IF BREAD_IN_PANTRY == 0: 
            RETURN TRUE
        ELSE:
            RETURN FALSE
    ELSE:
        IF TEMPERATURE_F >= 30:
            RETURN TRUE
        ELSE:
            RETURN FALSE
```

## Forests

While decision trees are very approachable, they are far from a state-of-the-art machine learning model. Some of the most obvious downsides:

1. They are best at modeling relationships in the data that have clear split points (something very different happens above 50 degrees compared to below), but poor at modeling linear relationships (for every 1 degree increase in temperature, bike ridership increases by 20). 
2. Any decision tree that is compact enough to be easily interpretable will have a relatively small number of decision splits (max $2^{n} - 1\text{ where n = number of layers}$, or 31 for 5 layers). Interpretable decision trees are often not complex enough to capture much of the signal in a dataset.
3. Letting decision trees get "deep" (many layers) inevitably leads to the opposite problem, overfitting, in which the tree captures irrelevant information. For example, once the number of datapoints in a particular branch is under 10, the tree could split based on temperature = 34.325 vs 34.266, gaining accuracy by categorizing meaningless minutiae instead of finding important differences between datapoints. 

However, decision trees have proven far more powerful when we use many of them at once, combined into a "forest". In a forest or ensemble model, many decision trees are "grown" from an optimal-split-finding algorithm run on the data, usually with different initial conditions or restrictions (e.g. only use this subset of features) so that the trees look pretty different. The trees then "vote" in some manner on the correct prediction (for example by taking the average of their predictions). 

While there are many different implementations of this type, the high-level intution is that:

1. The large number of trees (typically 100+) creates a much more complex model that can capture more signal from data and make better predictions. Some trees might "specialize" in certain data subsets.
2. The "voting" aggregation limits one overfit tree from influencing the outcome too much. Broadly, specious inferences may occur in many trees but they will all be different and so cancel out, whereas meaningful inferences will occur in at least some significant portion of trees, and so "bubble up" out of the random soup.

## Gradient Boosting
Gradient Boosting is a clever way of assembling a forest. To simplify greatly, imagine building a single decision tree, then using it to make a prediction. While not very accurate, you will get both a prediction and an error term for each datapoint. The error term is simply how far off the prediction was. Now, build another tree with the purpose not of predicting the datapoint's actual outcome, but rather the error that we just calculated. Say that we could then predict this error with 100% accuracy. So what? We can combine the predictions of the two trees to perfectly recapture the outcome we are trying to predict! In essence, we can build a forest of two models, sum their outputs, and have 100% accuracy. 

In reality, of course, the second model won't have total success in predicting the error. But it will almost certainly capture some of the signal that the first tree missed. So our aggregate model is almost certainly better than the first tree alone. 

Now note that we can continue this process indefinitely, incrementally improving our prediction with each new tree.

This is a highly simplified version of actual algorithms in the Gradient Boosting family (and particularly the current king, XGBoost), but it gets the point across. For our purposes, the very neat thing about these models is that

1. They are powerful and work very well for problems with tabular data, and
2. However complex and clever the process for generating the trees, the end result is simply a collection, of some size, of relatively simple trees. In other words, if we understand the functions of the individual trees, and the method for aggregating them (in this case, a simple sum of their datapoint-wise predictions) we can treat the actual XGBoost algorithm or whatever we are using as a black box. 

This makes them ideal candidates (among complex models) for visualization and interpretation. If you are interested in the details of gradient boosting, try [this article](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/) or [this one](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/).

## Detailed description of our model/problem
For the remainder of this notebook, we're going to use this example, so let's describe some parameters of our model. The model is a Gradient Boosting Regressor trained on 17,379 rows of the bike dataset, using the 11 columns presented above. The objective is to predict the number of riders for a particular hour. This is a *regression* problem, meaning that we are trying to predict a quantity, rather than a yes/no. Our particular gradient boosting model uses 300 trees and is limited to depth 2. 

It has an $R^2$ score of 0.906 on on the training set (70/30 split) and 0.901 on the test set. $R^2$ is a measure of performance for regression models, based on the squared error of the predictions as compared to the ground truth. At 1, the predictions are always perfect, and at 0, the predictions are only as good as predicting the mean of the output each time. $R^2$ can be arbitrarily negative if your predictions are worse than this.

So a 0.9 $R^2$ is decent, but not spectacular. The histogram below shows how these errors are distributed.

In [ ]:
#kick the data out to a file first so we can get around the 5000 row max that Altair has
#no need to run this unless you've made modifications to the notebook
# pd.concat([
#     pd.Series(f2t.y-f2t.pred_y, name = "error"), 
#     pd.Series(abs(f2t.y-f2t.pred_y), name = "absolute_error")
# ], axis=1).to_json("notebook_resources/error_histogram_data.json", orient="index")
histogram_df = pd.concat([
    pd.Series(f2t.y-f2t.pred_y, name = "error"), 
    pd.Series(abs(f2t.y-f2t.pred_y), name = "absolute_error")
], axis=1)

In [ ]:
alt.Chart(histogram_df).mark_rect().encode(
    x = alt.X("error:Q", bin = alt.Bin(maxbins = 60), axis = alt.Axis(title = "Model errors")),
    y = "count()"
)

We can see that we're rarely off by more than 80, but can we make a more specific claim about how good this model is? Let's convert this to a cumulative histogram.

In [ ]:
alt.Chart(histogram_df).mark_rect().transform_joinaggregate(
    total_count="count(*)"
).transform_window(
    sort=[{'field': "absolute_error"}],
    frame=[None, 0],
    cumulative_count='count(*)'
).transform_calculate(
    Percent = "datum.cumulative_count / datum.total_count"
).encode(
    x = alt.X("absolute_error:Q", bin = alt.Bin(maxbins = 40), axis = alt.Axis(title = "Cumulative Model Errors")),
    y = alt.Y("Percent:Q", axis = alt.Axis(format = "%")),
    tooltip = [alt.Tooltip("Percent:Q", format="%"), alt.Tooltip("absolute_error:Q", title = "Error")]
)

So we can now see that about 85% of predictions are off by less than 80, and about 50% are off by less than 30.

You might reasonably wonder if it's possible to build a better model than this. And in fact, it is. The chart below shows the $R^2$ of Gradient Boosting Trees with depth 1-4 as they are grown from 1 to 300 trees each. Performance for both the train and test set is reported. Zoom in to see it better.

In [ ]:
effect_additional_trees = []
train_x, test_x, train_y, test_y\
= train_test_split(bike_dataset["x"], bike_dataset["y"], test_size = 0.3)
for estimators in range(5, 300, 20):
    for depth in range(1,5):
        model = GradientBoostingRegressor(
            n_estimators = estimators, 
            max_depth = depth, 
            learning_rate = 1
        )
        model.fit(train_x, train_y)
        effect_additional_trees.append(
            {
                "number of trees" : estimators,
                "depth" : depth, 
                "train" : r2_score(train_y, model.predict(train_x)),
                "test" : r2_score(test_y, model.predict(test_x))
            }
        )
effect_additional_trees_df = pd.DataFrame(effect_additional_trees)
effect_additional_trees_df = effect_additional_trees_df.melt(
    id_vars = ['number of trees', "depth"],
    var_name = 'set',
    value_name = 'r_squared'
)
effect_additional_trees_df["outcome"] = effect_additional_trees_df.apply(
    lambda x: "depth: " + str(x["depth"]) + ", set: " + x["set"],
    axis = 1
)

In [ ]:
additional_trees_lines = alt.Chart(effect_additional_trees_df).mark_line().encode(
    x = "number of trees:Q",
    y = "r_squared:Q",
    color = alt.Color("outcome", type="nominal", scale = alt.Scale(scheme = "category20"))
)

additional_trees_points = alt.Chart(effect_additional_trees_df).mark_point().encode(
    x = "number of trees:Q",
    y = "r_squared:Q",
    color = alt.Color("outcome", type="nominal", scale = alt.Scale(scheme = "category20")),
    tooltip = [alt.Tooltip("outcome"), alt.Tooltip("r_squared")]
)

(additional_trees_lines + additional_trees_points).interactive()

We can clearly see that the 1-deep model performs very poorly. Our 2-deep model takes a bit longer to achieve high performance, but at 200+ trees it's very close to the best performance. And, the performance of the train and test sets are nearly identical, while the training set for the 4-deep model performs over 0.05 better than its test set, indicating a degree of overfitting.

Likely, smarter hyperparameterization and feature engineering for the 4-deep model would probably produce a better predictor than our 2-deep naive attempt, but we can continue confident that our model has decent performance. As we'll see, we are going to leverage a unique feature of 2-deep decision trees to build our model explanations. 

## Explanations
How are we to actually understand a Gradient Boosting model? We need to generate an "explanation" for the model. This can take a lot of different forms - a list of facts or insights, a list of rules that the model follows, a chart, a visual analytics system, a mathematical or logical function, etc. Even the matrices where the model stores its low-level definition are explanations, just not very good ones. We are going to start with some of the more common/naive explanations and then develop some that work better.

Our first explanation is similar to the pseudocode function used in the toy "Lucas" example, but now applied to the first tree in our Gradient Boosting model.

Please note one change - most Gradient Boosting models assume a starting point of the dataset mean, from which point the first tree calculates the error (recall that this "prediction" results in an $R^2$ of 0). The mean in this case is 189.46 riders/hour. So every value below should be added or subtracted to that value, in order to get the actual prediction.

```
def one_tree_of_many():
    IF HOUR OF DAY <= 6:
        IF HOUR OF DAY <= 5: 
            RETURN -164.6
        ELSE:
            RETURN -113.4
    ELSE:
        IF TEMPERATURE_F <= 59.1:
            RETURN -13.5
        ELSE:
            RETURN 126.7
```

The pseudocode function presented above is... fine. The function is transparent, but all of these numbers and splits are hard to understand without context. Basically, we can see here that the model is trying to first reduce its error by predicting low ridership before 6 am, and after that, high ridership on 60+ degree days. We could even guess that 5 AM is the start of the morning commute for some, because that's where the negative effect begins to lessen.

All cool, but a lot to hold in our head at once. Clearly our recall is going to suffer if we're asked to hold more than a tree or two in mind at once. It's wholly impractical if there are 300 trees in the model. We can slightly condense and visualize into something flowcharty like this:

![Styled Flowchart Approach](notebook_resources/decision_flowchart.png)

This is easier to make sense of at a glance, but even with putting the splits into plain English and returning the actual prediction (rather than just the change from the mean), this approach will not scale well. And, it takes up a lot of space!

Finally, we could take a graphical approach. The chart below is a heatmap of the range of the Temperature and Hour of Day features, and color represents the change from the mean for datapoints in that region. You can hover to get exact values. This chart represents the exact same decison process that the pseudocode and flowchart above do.

In [ ]:
single_estimator_slider = wid.IntSlider(
    value = 0,
    min = 0,
    max = 300,
    step = 1,
    continuous_update = False,
    description = "Tree #"
)
ui = wid.HBox([single_estimator_slider])
output = wid.interactive_output(
    f2t.visualize_estimator,
    {
        "estimator_nums" : single_estimator_slider,
        "print_function_text" : wid.fixed(False)
    }
)
display(ui, output)

There's one big advantage of this approach: we can see the minimum and maximum of a feature's ranges, and so the "threshold" at each split makes more sense in context. The split <=50 degrees would have a different meaning in Florida in the summer vs. Alaska in the winter. In the former it is likely correlated with mornings and nights, whereas in the latter, it might represent a few outlier days. And this need for context will be more pronounced for features that have less inherent meaning to the analyst.

Before we move on, take the time to interact with this cell to get a better sense of the model we are interpreting. You can visualize any single tree in the model by moving the __Tree #__ slider. Note that some trees have one chart, and others two - if both splits in the 2nd layer use the same feature, then we can compactly represent the tree in only one. Look for patterns - for example, later trees generally have smaller "contributions" to the prediction. Why might this be, based on what we know about Gradient Boosting?

## Combining explanations
Still, this approach will not scale well. 300 charts are scarcely better than 300 paragraphs. However, let's take a look at an interesting feature of two of these trees.

In [ ]:
_ = f2t.visualize_estimator(8, print_function_text = False) 
_ = f2t.visualize_estimator(9, print_function_text = False)

These two trees use different split points, on the same features. So the charts we use to display them have the same X and Y axes, and thus the same actual squares in our heatmap. Therefore, we can explain both trees at once with a single chart, by overlaying them on top of each other. The resulting chart will display the sum of the predicted values for each square on the heatmap. 

In [ ]:
f2t.visualize_estimator([8,9], False, False, False)

Success! By doing this, we've halved the space required to display two trees, with only a slight increase in the complexity of the chart. In fact, I'd argue that it's easier to interpret the two together, because together they more fully describe how the model treats one particular set of variables. And, if we needed to switch to another chart between these two, we'd spend mental overhead on re-orienting ourselves to new axes.

How many charts it would take to visualize every tree in the model, if they were all combined where possible? At a maximum, with 12 features we have $(12*11)/2 = 66$ possible charts.

In [ ]:
f2t.explain(1.)
f2t.visualize_components(False)

That's 43 charts above, indicating that our model thinks many of the interactions (but not all), are important. You'll notice that these are sorted in descending order of "importance", e.g. the first few have wider color ranges, indicating that they affect the predictions more, whereas the later charts have lighter colors and generally small values. 

Let's take a second to quantify our savings in space and complexity - our model has 300 trees, many of which require 2 charts to represent. A quick calculation should tell us exactly how many charts we would need:

In [ ]:
num_charts_required = 0
for estimator_num in range(300):
    chart_components, _, _, _, _ = f2t._extract_components(False, [f2t.model.estimators_[estimator_num]], False)
    num_charts_required += len(chart_components)
print("{} charts are needed to show each tree individually".format(num_charts_required))

So we've reduced the number of charts from 460 down to 43, which is 9.3% of the original. Not bad! We also could have represented the model with 300 pseudocode blocks or 300 flowcharts.

However, 43 charts is still far too many. We can't yet call this "interpretable". 

And before we even get there, another problem occurs - how do we know that this process has worked correctly? How do we have confidence that these charts faithfully represent our underlying model?

## Model Fidelity

This question is actually going to go to the heart of what's presented here. We'd like to have some assurance that the explanation we extract from the model is actually close to the real thing. But what does this mean? How can we actually calculate fidelity?

One way would be to treat the explanation as a black box, and use it to make predictions from our dataset. We could then compare those predictions to the predictions from our model, and if they are the same, or reasonably close, then we can call the explanation "faithful". In fact, we can calculate the degree to which the explanation is faithful to the model using the $R^2$ metric from before. In essence, we just treat the explanation as a model, and the original model's output as our new training set. We'll return to this concept of an explanation as just a new model later.

But how do we use this explanation, this series of charts, to make a prediction? Well, because this is only lightly abstracted from the underlying decision trees, we can actually use the same method that the model does. 

```
For each datapoint:
    PREDICTION = mean riders/hour of 189.46. 
    For each chart:
        Identify the two features in this chart
        Get the datapoint's values for those two features
        Identify which square of the heatmap the datapoint will fall into based on those values
        PREDICTION += the prediction for that square (the value encoded as color)
    RETURN PREDICTION
```

This method is nigh-identical to the process that the Gradient Boosting model uses to make predictions, except that instead of using a function for each tree to output 300 individual predictions (which are then summed), we use the data structures underlying 43 charts, each of which is just a lookup table mapping feature values to predictions.

This is a little abstract, so let's look at how this works in practice. Below, I've overlaid a sample of datapoints from our dataset onto the Hour of Day/Temperature (F) chart (the first one of the 43 above).

In [ ]:
f2t.explain(.5)
f2t.visualize_components(True, 300)

This visualization shows the process of a datapoint "falling into" a cell of the heatmap. Therfore, the chart "outputs" this value for that datapoint, and the datapoint is added to the prediction.

We can also visualize this process from the point of view of a single datapoint. The visualization below is a parallel coordinates chart, in which each line represents a single datapoint, and each column shows its value on a particular feature. Here, the values are not the features of the datapoint itself (e.g. the weather was rainy this hour) but rather the prediction output by a particular chart (e.g. this datapoint fell into a +50 riders square on the "Hour of Day/Temperature(F)" chart). 

In [ ]:
f2t.explain(.95)
f2t.visualize_datapoints(
    cumulative = True,
    num_datapoints = 2,
    explanation_type = "minimal",
    color_encoding = "prediction",
    auto_display = False
)

This chart visualizes the prediction process for two data points. Note that they each start on the left at the rider/hour mean of 189.46. Then their prediction changes as we sum the output of each chart. As expected, the first few charts cause a major change in the predictions, but eventually the lines flatten out and more gradually approach the final prediction. In general, we see that the last charts don't matter as much. 

The last column is the actual prediction from our Gradient Boosting model, and so a little dip or rise at the end shows how close our explanation comes to recreating that prediction. 

While this should make sense as a method, we don't even need to understand it in order to assess the explanations's fidelity. It'll be enough to see that the $R^2$ is high when compared to the model's predictions. Let's do that.

In [ ]:
f2t.explain(1.)
print("Our explanation has an R-squared value of "\
+ "{} when compared to the predictions of the Gradient Boosting model"\
      .format(round(f2t.evaluate_explanation(),3))
)

We get a 0.98 $R^2$ score, which indicates that our explanation faithfully recreates the underlying Gradient Boosting model!

Where does the slight loss come from? One modification that we are making by moving from decision trees to heatmaps is that heatmaps divide each feature into regular ranges, producing the tiles or squares that we see. Because these divisions are regular, they won't capture the _exact_ threshold that the underlying tree uses. In other words, if we used finer divisions in our heatmap, the $R^2$ would approach 1.0. 

However, dividing an axis into 100 tiles takes way longer to calculate, so our current approach (40 tiles) will serve fine. 

## Parsimonious Explanations

We are still left with the problem of interpretability, given our 43 charts. However, we can leverage an insight from our parallel coordinates plot to reduce this. As we noticed above, charts vary widely in the range of their values and by extent, the degree to which they affect the average prediction. Could we remove some of the lower value charts with a minimal loss of fidelity? This would just produce another, shorter explanation.

Taking this idea a step further, we could actually construct an explanation chart-by-chart using the components of our full explanation, adding one chart at a time until the $R^2$ value hit a user-defined threshold. But how to choose which order to add charts in? We might assume that an effective proxy would be the mean absolute value of the values in a chart, but note that it's possible for a chart to have extreme high or low values in ranges where few datapoints fall. 

Because it's pretty fast to generate a prediction from a chart and compare it to the model prediction, we can actually just employ a greedy search method where we try every chart, select the one that generates the most improvement in $R^2$ value, and then repeat the process until the fidelity threshold is reached. The pseudocode for our algorithm looks like:

```
explanation = [empty array]
fidelity = 0
While fidelity <= fidelity_threshold
    Initial Prediction = 189.46 for each datapoint
    For chart in the 43 charts from the full explanation:
        If chart not in explanation:
            Generate prediction from this chart for each datapoint
            Add prediction to Initial Prediction and calculate R^2
    Identify the chart that produced the highest R^2
    Add that chart to the explanation
    Calculate the fidelity of the current explanation
```

This algorithm allows us to calculate the most parsimonious explanation that is accurate to the underlying model within a specified degree. We can modify this slightly to produce a graph of how the number of charts in the explanation changes with the fidelity threshold. This will give us a sense of the tradeoffs between succinctness and accuracy, which is a recurring question in interpretability.

In [ ]:
list_num_charts = []
for i in range(1,100, 1):
    fidelity = i/100.
    f2t.explain(fidelity)
    list_num_charts.append({
        "fidelity_threshold" : fidelity, 
        "num_charts" : len(f2t.explanation),
        "mean_absolute_error" : f2t.evaluate_explanation("mae")
    })

In [ ]:
num_charts_df = pd.DataFrame(list_num_charts)
num_charts_df = num_charts_df.melt(id_vars = "fidelity_threshold")
num_charts_df.head()

line = alt.Chart(num_charts_df).mark_line().encode(
    x = "fidelity_threshold:Q",
    y = "value:Q",
    color = "variable"
)

points = alt.Chart(num_charts_df).mark_point(size=4).encode(
    x = "fidelity_threshold:Q",
    y = "value:Q",
    color = "variable",
    tooltip = ["fidelity_threshold", "value"]
)

(line+points).properties(
    title = "Size of explanation required to meet various fidelity thresholds"
).interactive()

This visualization tells us something extraordinary! We can recreate our Gradient Boosting model to 0.9 $R^2$ with only 4 charts, and up to 0.95 $R^2$ with 7 charts. We can also see how the Mean Absolute Error of our predictions decreases as more charts are added. At 0.95 $R^2$, the average error for our predictions is less than 30. 

## Return to Automated Insight Extraction

Let's take a look at the visual explanation of our Gradient Boosting model at the 0.95 level.

In [ ]:
f2t.explain(0.95)
f2t.visualize_components(False)

This explanation, consisting of 7 charts and possessing a certificate of quantifiable fidelity, is manageable enough in size for us to develop some insights about the dataset from the charts, bringing us back at long last to our goal of automated insight extraction. 

Take a second to inspect the charts above and extract some patterns. Keep in mind that these are features of the model, not necessarily of the dataset. I notice the following:

1. Hour of Day is obviously an important feature, since it appears in every chart.
2. On work days, there are bursts in ridership from 5am - 9am and 5pm - 7pm, clearly corresponding to rush hour.
3. Weekends show a more gradual pattern, with low ridership in the early morning, rising throughout the day.
4. Light precipitation has a negative effect on ridership, with the effect being slightly stronger during rush hour.
5. Ridership rose in the 2nd year of the bike sharing program, particularly from 6am - 8pm. Ridership from 8pm - midnight declined slightly in the 2nd year.
6. 59 degrees seem to be the point at which ridership suddenly increases.

In addition, this visualization lets us do some high-level thinking about model complexity and interpretability. Say I notice that it's confusing to have both the Temperature (F) and Feels Like (F) features in the model. They appear in separate charts, and generally make the effect of temperature hard to see in one place. I can test the effect of removing the feature by training a new Gradient Boosting model without it, and generating a 0.95 $R^2$ explanation for that.

In [ ]:
f2tNoFeelsLike = ft.ForestForTheTrees(dataset = "bike", sample_size = None, num_tiles = 40,\
                                      quantiles = False, learning_rate = 1.)
feels_like_pos = f2tNoFeelsLike.feature_locs["Feels Like (F)"]
f2tNoFeelsLike.feature_locs = {key:val for key, val in f2tNoFeelsLike.feature_locs.items()\
                               if key != "Feels Like (F)"}
f2tNoFeelsLike.feature_names = [x for x in f2tNoFeelsLike.feature_names if x != "Feels Like (F)"]
f2tNoFeelsLike.x = np.hstack((f2tNoFeelsLike.x[:,:feels_like_pos], f2tNoFeelsLike.x[:,feels_like_pos+1:]))
f2tNoFeelsLike.build_base_model(300)
f2tNoFeelsLike.extract_components(True, False)
f2tNoFeelsLike.explain(.95, None)
f2tNoFeelsLike.visualize_components(False)

In [ ]:
print(
    "The new Gradient Boosting model has an R squared of {}".format(
        round(
            r2_score(
                f2tNoFeelsLike.pred_y, 
                f2tNoFeelsLike.y)
            ,3
        )
    )
)

This explanation uses only 5 charts to reach 0.95 $R^2$, and the underlying model has only slightly reduced $R^2$ when compared to the raw data - 0.895 versus 0.905 for our original model. Depending on circumstances such as use case, audience, etc. we could decide to use one versus the other. I personally like this one better since we get a clearer picture of the effect of temperature in one chart!

In fact, if our audience was unaware of the Gradient Boosting model, we could just present these 5 charts as the model we have built, and calculated the $R^2$ directly from these charts to the data.

In [ ]:
r_squared_against_raw_data = r2_score(
    f2tNoFeelsLike.predictions_base +\
    np.sum(
        np.array(
            list(
                f2tNoFeelsLike._get_prediction_contributions_by_key(
                    f2tNoFeelsLike.explanation_components,
                    f2tNoFeelsLike.explanation
                ).values()
            )
        ), 
        axis = 0
    ).reshape(-1,1),
    f2tNoFeelsLike.y
)

print("The 5-chart model above has an R-squared of {} when used to predict the outcomes from the raw data"\
.format(round(r_squared_against_raw_data,3)))

We certainly take a double penalty in our accuracy measure here, first from the Gradient Boosting model's loss compared to the raw data, and then again from the fact that we use a 0.95 $R^2$ explanation of that model. We could certainly increase the fidelity threshold of our explanation and get that number closer to the 0.895 $R^2$ ceiling. In either case, we can choose the explanation that fits the audience and circumstances.

## Some Additional Lenses into our Model

We can now return to our parallel coordinates chart from earlier and add some interactivity to explore this new model in depth.

The __View__ dropdown lets you see either the cumulative prediction (which converges) or the individual predictions contributed by each chart.

__Color By__ lets you pick a value of the datapoint (e.g. the humidity for that hour) and color by that feature. You can use this to find trends in how a chart converts features to predictions. You can also choose to color by the predicted value from the underlying model, or the loss (error) of the explanation's prediction compared to that of the model.

__# datapoints__ lets you select a random batch of 1-100 datapoints to visualize.

You can also zoom and pan, and click/shift+click to highlight single/multiple lines.

In [ ]:
#DON'T RUN THIS unless you have modified the code and need to cache data for a new vis
#otherwise this has already been stored as a file and will be loaded below
#cache output of the longer-running calculations so that visualizations can run more quickly
#f2tNoFeelsLike.cache_visualize_datapoints(True)
#f2tNoFeelsLike.cache_visualize_components(start = 1, end = 100, step = 1)
#f2tNoFeelsLike.load_cache_from_file()

In [ ]:
cumulative_selector = wid.Dropdown(
    options = [('Cumulative Prediction', True), ('Contributions by Feature', False)],
    value = True,
    description = 'View',
)

num_datapoints_slider = wid.IntSlider(
    value = 50,
    min = 1,
    max = 100,
    step = 5,
    description = '# datapoints',
    continuous_update = False,
    orientation = 'horizontal',
    readout = True,
    readout_format = 'd'
)

color_encoding_selector = wid.Dropdown(
    options = f2t.feature_names + ["prediction", "explanation_loss"],
    value = "prediction",
    description = 'Color By',
)

ui = wid.HBox([cumulative_selector, color_encoding_selector, num_datapoints_slider])

output = wid.interactive_output(
    f2tNoFeelsLike.visualize_datapoints,
    {
        "cumulative" : cumulative_selector,
        "num_datapoints" : num_datapoints_slider,
        "explanation_type" : wid.fixed("minimal"),
        "color_encoding" : color_encoding_selector,
        "auto_display" : wid.fixed(True)
    }
)

display(ui, output)

Lastly, if you want to get a sense of how our Gradient Boosting model was actually built, tree by tree, we can generate a single complete explanation for models trained with 1,2,3....100 trees and play it as a movie, to see how a model of the dataset is iteratively assembled. 

Click __Play__ or move the slider to a particular number of trees to see that state.
Each frame of the movie will add a small dot to each square that changed since the previous iteration.

In [ ]:
f2tNoFeelsLike.load_cache_from_file()
play_control = wid.Play(
    interval = 5000,
    value = 1,
    min = 1,
    max = 100,
    step = 1,
    description = "Press play"
)
play_slider = wid.IntSlider(
    value = 1,
    min = 1,
    max = 100,
    step = 1,
    continuous_update = False,
    description = "# of trees"
)
wid.jslink((play_control, 'value'), (play_slider, 'value'))
ui = wid.HBox([play_control, play_slider])
output = wid.interactive_output(
    f2tNoFeelsLike.play_components,
    {"cache_id" : play_control}
)
display(ui, output)

There are a lot of neat insights into the model-building process here. I like watching how one chart, for example Hour of Day/Temperature(F), gradually builds up a gradient out of multiple individual trees. I remarked back in the Decision Trees section that individual trees couldn't model linear relationships well, but the Gradient Boosting model learns this relationship by assembling a set of trees with a graduated series of splits. 

## Final Thoughts

In this notebook, we've investigated a method for probing the internals of Gradient Boosting models and constructing a usable explanation. I hope that both the product (a method for generating a succinct explanation) and the process (treating explanations as structures which can be distilled from models, manipulated, and reasoned about) are helpful. To summarize the main contributions:

1. Gradient Boosting models for regression can be manipulated into a visual explanation that supports insight extraction and sensemaking.
2. This explanation comes with a quantifiable guarantee of fidelity to the underlying model. Moreover, we can manipulate the explanation to trade off between complexity and fidelity with the aim towards finding an appropriate explanation for a given situation.

Beyond this, the work here lays the foundation for additional advances in model interpretability. Consider the fact that in generating an explanation of this type, we have actually distilled a new model, which can be used for making predictions itself. This model is somewhat unique, though, in that it doesn't consist of a function so much as a lookup table of values. In a sense, it is a visual model rather than a typical model which has been visualized through some post-processing method. By considering the explanation in this light, we can potentially do the following:

1. Generate explanations for important data subsets. A "local" explanation of this type might not only be shorter, but could reveal important features for that subset. If 10 charts were required to meet the 0.95 threshold, then we could claim that it is "harder" to make accurate predictions for that subset.
2. Create a fully manipulable complex model! It's relatively trivial to develop a visual interface that allows a user to change the value of any square of any chart. The question is how to develop a visual analytics system that allows the user to make that choice meaningfully. A trivial example would be correcting squares in low-density regions of the dataset, which may have extreme values. This could be used to inoculate the model against adversarial probing or ensure that data falls within desired ranges. 
3. These visual models could be an interesting extension to highly interpretable scoring systems (such as [this](https://link.springer.com/article/10.1007/s10994-015-5528-6)) developed for by humans in medical systems. With a low number of tiles, one of these models could be placed in a reference card or a simple app to allow for interpretable, powerful risk assessments. The visual format also innately supports reasoning about if=>then conjectures, because a user can see clearly how a change in a particular value would affect the overall prediction (similar to work into [Partial Dependence Bars](https://josuakrause.github.io/info/material/chi2016-prospector.pdf)). 
4. Lastly, it is worthwhile to investigate how this approach could be used to represent tree ensembles with depth 3 or higher. The only current limitation is the heatmap visualization and the general difficulty of representing 3+ dimensional distributions. An interesting challenge for the vis community!

Thank you for taking the time to read and interact with this notebook!